I import pandas so I can read the CSVs into dataframes

In [1]:
import pandas as pd
import numpy as np

I have pandas convert my csv's into dataframes

In [2]:
invoice_test = pd.read_csv('invoice_test.csv', low_memory=False)
invoice_train = pd.read_csv('invoice_train.csv', low_memory=False)
client_test = pd.read_csv('client_test.csv', low_memory=False)
client_train = pd.read_csv('client_train.csv', low_memory=False)
sample_submission = pd.read_csv('SampleSubmission.csv', low_memory=False)

print(invoice_test.shape, invoice_train.shape, client_test.shape, client_train.shape)

(887059, 16) (864499, 16) (58069, 5) (135493, 6)


In [4]:
client_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   disrict        135493 non-null  int64  
 1   client_id      135493 non-null  object 
 2   client_catg    135493 non-null  int64  
 3   region         135493 non-null  int64  
 4   creation_date  135493 non-null  object 
 5   target         135493 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 6.2+ MB


In [5]:
def feature_change(cl, inv):

    cl['client_catg'] = cl['client_catg'].astype('category')
    cl['disrict'] = cl['disrict'].astype('category')
    cl['region'] = cl['region'].astype('category')
    cl['region_group'] = cl['region'].apply(lambda x: 100 if x<100 else 300 if x>300 else 200)
    cl['creation_date'] = pd.to_datetime(cl['creation_date'])

    inv['counter_type'] = inv['counter_type'].map({"ELEC":1,"GAZ":0})
    inv['counter_statue'] = inv['counter_statue'].map({0:0,1:1,2:2,3:3,4:4,5:5,769:5,'0':0,'5':5,'1':1,'4':4,'A':0,618:5,269375:5,46:5,420:5})
    
    inv['invoice_date'] = pd.to_datetime(inv['invoice_date'], dayfirst=True)
    
    return cl, inv

In [6]:
client_train1, invoice_train1 = feature_change(client_train, invoice_train)
client_test1, invoice_test1 = feature_change(client_test, invoice_test)

C:\Users\Nick_\AppData\Local\Temp\ipykernel_121224\3295624376.py:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  cl['creation_date'] = pd.to_datetime(cl['creation_date'])
C:\Users\Nick_\AppData\Local\Temp\ipykernel_121224\3295624376.py:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  cl['creation_date'] = pd.to_datetime(cl['creation_date'])


In [7]:
train_df1 = pd.merge(client_train1,invoice_train1, on='client_id', how='left')

test_df1 = pd.merge(client_test1,invoice_test1, on='client_id', how='left')

print(train_df1.columns)

Index(['disrict', 'client_id', 'client_catg', 'region', 'creation_date',
       'target', 'region_group', 'invoice_date', 'tarif_type',
       'counter_number', 'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'old_index',
       'new_index', 'months_number', 'counter_type'],
      dtype='object')


In [8]:
train_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 973947 entries, 0 to 973946
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   disrict               973947 non-null  category      
 1   client_id             973947 non-null  object        
 2   client_catg           973947 non-null  category      
 3   region                973947 non-null  category      
 4   creation_date         973947 non-null  datetime64[ns]
 5   target                973947 non-null  float64       
 6   region_group          973947 non-null  int64         
 7   invoice_date          864499 non-null  datetime64[ns]
 8   tarif_type            864499 non-null  float64       
 9   counter_number        864499 non-null  float64       
 10  counter_statue        864499 non-null  float64       
 11  counter_code          864499 non-null  float64       
 12  reading_remarque      864499 non-null  float64       
 13 

In [9]:
def drop(df):

    col_drop = ['client_id', 'creation_date']
    for col in col_drop:
        df.drop([col], axis=1, inplace=True)
    return df

In [10]:
train_df1 = drop(train_df1)
test_df1 = drop(test_df1)

In [11]:
from pycaret.classification import *

In [ ]:
reg = setup(train_df1, target = 'target', fold_shuffle=True, remove_multicollinearity = True, multicollinearity_threshold = 0.95, imputation_type="iterative", silent = True)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:53:15
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


I run 'compare_models' to have pycaret run the data through various algorithms

In [ ]:
best = compare_models()

In [ ]:
plot_model(best)

In [ ]:
evaluate_model(best)

In [ ]:
best

In [ ]:
tuned_model=tune_model(best)

In [ ]:
final_model = finalize_model(tuned_model)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
test.insert(36, "region 199", 0)

In [ ]:
test.replace([np.inf, -np.inf], np.nan, inplace=True)
test = test.fillna(0)

In [ ]:
predictions = predict_model(final_model, data=test)
predictions

In [ ]:
submission = pd.DataFrame({
        "client_id": sample_submission["client_id"],
        "target": predictions['Label']
    })
submission.to_csv('submission.csv', index=False)